# Import


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!pip install plotly_express

In [4]:
!pip install geopandas

In [5]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'

 trips_2019_10_10.csv	    trips_2019_3_26.csv        trips_2019_6_5.csv
 trips_2019_10_11.csv	    trips_2019_3_27.csv        trips_2019_6_6.csv
 trips_2019_10_12.csv	    trips_2019_3_28.csv        trips_2019_6_7.csv
 trips_2019_10_13.csv	    trips_2019_3_29.csv        trips_2019_6_8.csv
 trips_2019_10_14.csv	    trips_2019_3_2.csv	       trips_2019_6_9.csv
 trips_2019_10_15.csv	    trips_2019_3_30.csv        trips_2019_7_10.csv
 trips_2019_10_16.csv	    trips_2019_3_31.csv        trips_2019_7_11.csv
 trips_2019_10_17.csv	    trips_2019_3_3.csv	       trips_2019_7_12.csv
 trips_2019_10_18.csv	    trips_2019_3_4.csv	       trips_2019_7_13.csv
 trips_2019_10_19.csv	    trips_2019_3_5.csv	       trips_2019_7_14.csv
 trips_2019_10_1.csv	    trips_2019_3_6.csv	       trips_2019_7_15.csv
 trips_2019_10_20.csv	    trips_2019_3_7.csv	       trips_2019_7_16.csv
 trips_2019_10_21.csv	    trips_2019_3_8.csv	       trips_2019_7_17.csv
 trips_2019_10_22.csv	    trips_2019_3_9.csv	       trips_2019_7

In [5]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'
way_id_on_trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

In [8]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.2'

1_road_network_use_segment.ipynb	    6_train_new_label.ipynb
2_road_network_mapping_with_ckd_tree.ipynb  DB_test_way_eps0.02.pkl
3_test_quickbundle_lebel.ipynb		    HDB_test_way_eps0.02.pkl
4_test_quickbundle_model_training.ipynb     QBcluster_test_way_treshold12.pkl
5_new_label.ipynb			    QBcluster_test_way_treshold24.pkl


## get trip data

In [53]:
dfs = []
day_len = []
day_trip_count = []
# days = range(1,32)
days = range(1,2)
# months = range(1,4)
months = range(1,2)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(way_id_on_trips_path + f'/lebeled/testtrips_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

# df = df[['trip_id', 'way_id', 'day', 'month','route_cluster']]
df.head()

[12599]
[356]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,route_cluster,month,day
0,2019-01-01 00:00:39,016000200000352253062499073,17.49378,102.79729,0,6.0,1,686967969,0.015018,1.0,1,1
1,2019-01-01 04:26:59,016000200000352253062499073,17.53611,102.75656,53,6.0,1,686957536,0.004565,9.0,1,1
2,2019-01-01 04:29:39,016000200000352253062499073,17.54386,102.73553,54,6.0,1,686912257,0.001649,10.0,1,1
3,2019-01-01 04:32:39,016000200000352253062499073,17.55281,102.71174,56,6.0,1,686911019,0.011723,10.0,1,1
4,2019-01-01 04:35:59,016000200000352253062499073,17.56263,102.68405,54,6.0,1,686910206,0.014603,10.0,1,1


## get pickle

In [54]:
!pip install hdbscan

In [55]:
from hdbscan import HDBSCAN 
import pickle
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.2'
pkl_filename = train_path+"/HDB_test_way_eps0.02.pkl"
with open(pkl_filename, 'rb') as file:
    clustering = pickle.load(file)
clustering.labels_

array([164, 193,  -1, ..., 707, 707, 707])

## get way segment labeled

In [56]:
way_segment_labeled_df = pd.read_csv(way_id_on_trips_path + f'/lebeled/HDB_label_esp0.02.csv')
way_segment_labeled_df['road_seg'] = way_segment_labeled_df['road_seg'].map(lambda x: eval(x))
way_segment_labeled_df

,way_id,road_seg,mid_lat,mid_lon,count,label
0,686967969,"(35, 449)",17.478832,102.795843,12,164
1,686957536,"(2895, 4520)",17.535332,102.761058,1,193
2,686912257,"(4519, 4520)",17.544479,102.734002,3,-1
3,686911019,"(4516, 4519)",17.556765,102.700705,3,149
4,686910206,"(4516, 4517)",17.566924,102.670093,6,149
...,...,...,...,...,...,...
56106,123076908,"(439, 442)",16.631444,102.800829,84,707
56107,123076908,"(414, 439)",16.616631,102.805282,129,707
56108,123076908,"(414, 562)",16.607361,102.812563,129,707
56109,91540198,"(419, 420)",16.678062,102.804382,49,707


# get rid of mini cluster and map labeled

In [59]:
map_way_label = []
for way_id in df['way_id'].to_list():
  label_count = way_segment_labeled_df[way_segment_labeled_df.way_id==way_id][['label', 'count']].sort_values(by='count')
  map_way_label.append([way_id, label_count.iloc[0].label])
map_way_label = pd.DataFrame(map_way_label, columns=['way_id', 'label'])
map_way_label

,way_id,label
0,686967969,164.0
1,686957536,193.0
2,686912257,-1.0
3,686911019,149.0
4,686910206,149.0
...,...,...
12594,879405396,707.0
12595,123076908,707.0
12596,143803013,707.0
12597,123076908,707.0


In [67]:
map_way_label.way_id.unique

<bound method Series.unique of 0        686967969
1        686957536
2        686912257
3        686911019
4        686910206
           ...    
12594    879405396
12595    123076908
12596    143803013
12597    123076908
12598     91540198
Name: way_id, Length: 12599, dtype: int64>

In [66]:
df.way_id.unique

<bound method Series.unique of 0        686967969
1        686957536
2        686912257
3        686911019
4        686910206
           ...    
12594    879405396
12595    123076908
12596    143803013
12597    123076908
12598     91540198
Name: way_id, Length: 12599, dtype: int64>

In [51]:
df_labeled = df
# df_labeled = df_labeled.merge(map_way_label, left_on='way_id', right_on='way_id', how='left')
df_labeled['label'] = map_way_label['label']
df_labeled

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,route_cluster,month,day,label
0,2019-01-01 00:00:39,016000200000352253062499073,17.493780,102.797290,0,6.0,1,686967969,0.015018,1.0,1,1,164
1,2019-01-01 04:26:59,016000200000352253062499073,17.536110,102.756560,53,6.0,1,686957536,0.004565,9.0,1,1,193
2,2019-01-01 04:29:39,016000200000352253062499073,17.543860,102.735530,54,6.0,1,686912257,0.001649,10.0,1,1,-1
3,2019-01-01 04:32:39,016000200000352253062499073,17.552810,102.711740,56,6.0,1,686911019,0.011723,10.0,1,1,149
4,2019-01-01 04:35:59,016000200000352253062499073,17.562630,102.684050,54,6.0,1,686910206,0.014603,10.0,1,1,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12594,2019-01-01 05:30:33,02800040000000GSNF610300028,16.596032,102.818230,70,7.0,441,879405396,0.001989,748.0,1,1,707
12595,2019-01-01 05:31:33,02800040000000GSNF610300028,16.604758,102.814772,63,7.0,441,123076908,0.003414,748.0,1,1,707
12596,2019-01-01 05:34:33,02800040000000GSNF610300028,16.628225,102.799135,58,7.0,441,143803013,0.001789,748.0,1,1,707
12597,2019-01-01 05:36:33,02800040000000GSNF610300028,16.646123,102.800078,68,7.0,441,123076908,0.000282,748.0,1,1,707


In [ ]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

# for way_id in pd.unique(mid_way_df['way_id']):
#   way = mid_way_df[mid_way_df['way_id']==way_id]

# plot_way = df_labeled[mid_way_df['label']!=-1]
plot_way = df_labeled

plot_way = plot_way.merge(way_segment_labeled_df[['way_id', 'mid_lat', 'mid_lon']], how="left")
plot_way['label'] = plot_way['label'].astype('category')
print(plot_way.shape)

# fig = px.line_mapbox(
fig = px.scatter_mapbox(
  plot_way, 
  lat="mid_lat", 
  lon="mid_lon",
  color="label",
  hover_data= ['way_id', 'mid_lat', 'mid_lon', 'label'], 
  # zoom=9, 
  width=1200, 
  height=800
)
fig.show()

(5895851, 15)


# pomegranate

In [11]:
state_names = []
uniq_label = df_labeled.label.unique()
for label in uniq_label:
  state_names.append(str(label))
state_names[:5]

X = []
labels = []
trip_df = df_labeled
for month in months:
  for day in days:
    day_df = trip_df[(df.day == day) & (trip_df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.way_id.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.label.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

['686967969' '686957536' '686912257' '686911019' '686910206']
['164', '193', '-1', '149', '149']
356
356


In [16]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

['686967969' '686957536' '686912257' '686911019' '686910206']
['817644082' '81349795' '575226821' '160728583' '160728584']
['164', '193', '-1', '149', '149']
['559', '-1', '383', '383', '-1']


In [12]:
from collections import Counter
import pickle

# road_df = pd.read_csv(road_data_path+"/route_network_with_cluster.csv")

# cluster_df = road_df[['cluster', 'center_lat', 'center_lon']]
# cluster_df = cluster_df.drop_duplicates()
# cluster_df.rename(columns = {'cluster':'sp', 'center_lat':'lat', 'center_lon':'lon'}, inplace = True)

way_seq = df_labeled.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df_labeled = df_labeled.loc[not_stop_mask]

way_df = df_labeled[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')

# loc_df = df[['trip_id', 'way_id', 'month','day']].merge(way_df,how='left')
# print(sp_seq_df.isna().any())
# loc_df

sp_list = set(map(str, way_df['way_id'].tolist()))

def score(model, seq, label):
  p = []
  prob,path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  
  # c = Counter(p)
  # tp = 0
  # if c[label]:
  #   tp = c[label]
  # total = sum(c.values())
  
  tp = 0
  for _state_name, _label in zip(p, label):
    if _state_name == _label:
      tp+=1
  total = len(label)


  # print("best prob : {}% ".format(np.exp(prob)*100))
  # print("beat path : {}".format(p))   
  # print("labels : {}".format(label))
  # print("sequen    : {}".format(seq))

  return tp/total

In [13]:
!pip install pomegranate

In [17]:
from pomegranate import *

total = len(X2)
fold = int(total/10)
start = 0
starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
for i in range(10):
  starts.append(start)
  start+=fold

accuracy_train = []
accuracy_test = []
 
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  X_test = X2[start:start+fold]
  labels_train = labels2[:start]+labels2[start+fold:]
  labels_test = labels2[start:start+fold]

  # บรรทัดนี้ปลอดภัยไว้ก่อน
  uniq_sp = set()
  for seq in X_train:
    for sp in seq:
      uniq_sp.add(sp)
  uniq_label = set()
  # for label in labels_train:
  for label in labels_train:
    for uniq in list(set(label)):
      uniq_label.add(uniq)

  new_label = []
  for label2 in labels_train:
    new_label.append(['None-start']+label2)
  labels_train = new_label
  
  unseen_sp = sp_list - uniq_sp
  print(len(unseen_sp))
  X_train.append(np.array(list(unseen_sp)))
  labels_train.append(['-1'] * len(unseen_sp))
  # labels_train.append(['None-start'] + ['-1'] * len(unseen_sp))

  # uniq_label.add('-1')

  model = HiddenMarkovModel.from_samples(
      DiscreteDistribution, 
      n_components=len(uniq_label), 
      X=X_train, 
      labels=labels_train,
      algorithm='labeled',
      state_names=list(uniq_label), 
      inertia=0.001,
      max_iterations=10,
      n_jobs=-1
    ) 

  accuracy_train_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_train, labels_train):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    train_score = accu/len(X_train)
    accuracy_train_percent.append(train_score)
  accuracy_train.append(accuracy_train_percent)

  accuracy_test_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_test, labels_test):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    test_score = accu/len(X_test)
    accuracy_test_percent.append(test_score)
  accuracy_test.append(accuracy_test_percent)

  print(f'Score {start} - {start+fold}')
  print(f'Train score')
  print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
  print(f'Test score')
  print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_test_percent[3]}')

print('all score')
print('train score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_train:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')

print('test score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_test:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')

53
Score 0 - 35
Train score
	25% trip traverse : 1.0
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
Test score
	25% trip traverse : 0.9714285714285714
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
62
Score 35 - 70
Train score
	25% trip traverse : 1.0
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
Test score
	25% trip traverse : 0.9428571428571428
	50% trip traverse : 0.9714285714285714
	75% trip traverse : 0.9714285714285714
	90% trip traverse : 0.9714285714285714
62
Score 70 - 105
Train score
	25% trip traverse : 1.0
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
Test score
	25% trip traverse : 0.9142857142857143
	50% trip traverse : 0.9714285714285714
	75% trip traverse : 1.0
	90% trip traverse : 1.0
40
Score 105 - 140
Train score
	25% trip traverse : 1.0
	50% trip traverse : 1.0
	75% trip traverse : 1.0
	90% trip traverse : 1.0
Test score
	25% trip traverse : 0.8
	50% tri

In [24]:
print("25%", end='\n')
print("50%", end='\n')
print("75%", end='\n')
print("90%", end='\n')

25%
50%
75%
90%
